In [141]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Latex, Math

# E2.4
## a)

In [100]:
def eq_disp(varstring, expr):
    display(Latex(f"${varstring}={sp.latex(expr)}$"))
s, t = sp.symbols('s, t')
R = 1/s

The laplace transform of a unit step function is
$$F(s)=\frac{1}{s}$$
The output is given by:
$$Y(s)=G(s)R(s)$$
The transfer function $G(s)$ is given by

In [101]:
G = sp.factor(4*(s + 50)/(s**2 + 30*s + 200))
p, q = G.as_numer_denom()
G

4*(s + 50)/((s + 10)*(s + 20))

To find $Y(s)$ we need the partial fraction expansion of $G(s)R(s)$

We can construct the partial fractions according to:
$$G(s)R(s)=\frac{K_{s 1}}{s+s_1}+\frac{K_{s 2}}{s+s_2}+\cdots+\frac{K_{s n}}{s+s_n}$$

First solve for the n poles 

In [102]:
Y = G*R
p, q = Y.as_numer_denom()
poles = sp.solve(q,s)
for i, pole in enumerate(poles):
    eq_disp(f'-s_{i}', pole)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Now we find the numerators $K_{s_i}$ 

In [103]:
K = []
for pole in poles:
    K.append(sp.simplify(p/q*(s-pole)).subs(s, pole))

In [118]:
p_fracs = [K[i]/(s-poles[i]) for i in range(len(K))]
Y = sum(p_fracs)
eq_disp('Y(s)',Y)

<IPython.core.display.Latex object>

We can check the validity of the partial fractions by comparing their sum to $G(s)R(s)$

In [113]:
sp.factor(G*R) == sp.factor(sum(p_fracs))

True

We can then transform to time domain to obtain $y(t)$

In [130]:
p_fracs_t = []
for frac in p_fracs:
    p_fracs_t.append(sp.inverse_laplace_transform(frac,s, t))
y = sum(p_fracs_t)
eq_disp('y(t)', y)

<IPython.core.display.Latex object>

where $\theta(t)$ is the heaviside function or unit step function

## b)
To get the final value of $y(t)$ we can set the unitstep function to 1

In [136]:
eq_disp('y(\infty)', y.subs(sp.Heaviside(t), 1))

<IPython.core.display.Latex object>

# 2.25

In [138]:
a, x = sp.symbols('a, x')
x0 = 0.6
y = a*x**3
eq_disp('y', y)

<IPython.core.display.Latex object>

The linear approximation is obtained from the first order taylor expansion of the amplifier function at the operating point 

In [140]:
y_lin = y.subs(x, x0) + y.diff(x).subs(x, x0)*(x-x0)
eq_disp('y_{linear}', y_lin)

<IPython.core.display.Latex object>

# 2.31

# 2.26